# Eulero

- Eulero ("Graeco-Latin Square", "Euler Square") is a puzzle with letters and digits. The goal is to fill in the grid with symbols according to the following rules:

- Each cell contains one letter and one digit.
- Every row and every column contains each letter and each digit exactly once.
- No two cells contain the same pair of symbols.

In [22]:
def readGrid(path):
    with open(f"../assets/data/Eulero/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("problems/26_5x5")
    print(m, n)
    print(len(grid), len(grid[0]))
    print(grid)
    

5 5
5 5
[['00', '01', '00', '00', '00'], ['10', '00', '00', '00', '04'], ['00', '00', '00', '20', '00'], ['00', '50', '01', '33', '40'], ['00', '00', '00', '00', '02']]


In [24]:
import numpy as np
from ortools.sat.python import cp_model as cp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.pyplot import MultipleLocator

def eulero_solver(X, Y, grid):
    model = cp.CpModel()
    solver = cp.CpSolver()
    x = {}
    y = {}
    model = cp.CpModel()
    solver = cp.CpSolver()
    for i in range(X):
        for j in range(Y):
            x[0, i, j] = model.NewIntVar(1, X, f"x_0_{i}_{j}")
            x[1, i, j] = model.NewIntVar(1, X, f"x_1_{i}_{j}")
    for i in range(X):
        for j in range(Y):
            nums = list(grid[i][j])
            a_, b_ = int(nums[0]), int(nums[1])
            if a_ != 0:
                model.Add(x[0, i, j] == a_)
            if b_ != 0:
                model.Add(x[1, i, j] == b_)
    
    for i in range(X):
        rows1 = [x[0, i, j] for j in range(Y)]
        rows2 = [x[1, i, j] for j in range(Y)]
        model.AddAllDifferent(rows1)
        model.AddAllDifferent(rows2)
    
    for j in range(Y):
        cols1 = [x[0, i, j] for i in range(Y)]
        cols2 = [x[1, i, j] for i in range(Y)]
        model.AddAllDifferent(cols1)
        model.AddAllDifferent(cols2)

    uniqueness = []
    for i in range(X):
        for j in range(Y):
            y[i, j] = model.NewIntVar(( X + 2), (X + 1) * (Y + 1), f"y_{i}_{j}")
            model.Add(y[i, j] == x[0, i, j] * (X + 1) + x[1, i, j])
            uniqueness.append(y[i, j])
    
    model.AddAllDifferent(uniqueness)
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        for i in range(X):
            for j in range(Y):
                print(f"{solver.Value(x[0, i, j])}{solver.Value(x[1, i, j])}", end=" ")
            print()
    else:
        print("NOT FOUND!")
        
if __name__ == "__main__":
    m, n, grid = readGrid("problems/26_5x5")
    eulero_solver(m, n, grid)

# 55 12 33 24 41
# 34 43 15 51 22
# 42 54 21 13 35
# 23 31 52 45 14
# 11 25 44 32 53

34 21 55 42 13 
15 43 32 51 24 
53 12 44 25 31 
22 54 11 33 45 
41 35 23 14 52 
